In [31]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn import metrics
from sklearn.preprocessing import StandardScaler

from imblearn.over_sampling import ADASYN
data = pd.read_csv('./Data/featuresfloatv3.csv')

# x=data.loc[:,['statuses' , 'date_joined' , 'most_recent_post' , 'following' , 'followers' , 'likes', 'retweet' , 'retweeted_count'  ,'avg_tweets_by_hour_of_day', 'avg_tweets_by_day_of_week']]
x=data.iloc[:, :-1]
y = data.account_type.values.tolist()
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42,stratify=y)

sm = ADASYN(random_state=42)
x_train,y_train = sm.fit_resample(x_train,y_train)

# scale features
# Create an instance of StandardScaler
scaler = StandardScaler()
# Fit the scaler to your data
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)


# RandomizedSearchCV

In [23]:
RandomForestClassifier().get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'sqrt',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

Specify the range of values for some important hyperparameters

In [24]:
n_estimators_list = list(range(10,220,50))
criterion_list = ['gini', 'entropy']
max_depth_list = list(range(5,41,10))
max_depth_list.append(None)
min_samples_split_list = [x/1000 for x in list(range(5, 41, 10))]
min_samples_leaf_list = [x/1000 for x in list(range(5, 41, 10))]
max_features_list = ['sqrt', 'log2']
params_grid = {
    'n_estimators': n_estimators_list,
    'criterion': criterion_list,
    'max_depth': max_depth_list,
    'min_samples_split': min_samples_split_list,
    'min_samples_leaf': min_samples_leaf_list,
    'max_features': max_features_list
}

num_combinations = 1
for k in params_grid.keys(): num_combinations *= len(params_grid[k])

print('Number of combinations = ', num_combinations)
params_grid

Number of combinations =  1600


{'n_estimators': [10, 60, 110, 160, 210],
 'criterion': ['gini', 'entropy'],
 'max_depth': [5, 15, 25, 35, None],
 'min_samples_split': [0.005, 0.015, 0.025, 0.035],
 'min_samples_leaf': [0.005, 0.015, 0.025, 0.035],
 'max_features': ['sqrt', 'log2']}

In [25]:
def my_roc_auc_score(model, x, y): return metrics.roc_auc_score(y, model.predict(x))

model_rf = RandomizedSearchCV(estimator=RandomForestClassifier(class_weight='balanced'),
                              param_distributions=params_grid,
                              n_iter=100,
                              cv=3,
                              scoring=my_roc_auc_score,
                              return_train_score=True,
                              verbose=2)

model_rf.fit(x,y)

Fitting 3 folds for each of 100 candidates, totalling 300 fits
[CV] END criterion=gini, max_depth=5, max_features=sqrt, min_samples_leaf=0.015, min_samples_split=0.005, n_estimators=160; total time=   2.2s
[CV] END criterion=gini, max_depth=5, max_features=sqrt, min_samples_leaf=0.015, min_samples_split=0.005, n_estimators=160; total time=   2.2s
[CV] END criterion=gini, max_depth=5, max_features=sqrt, min_samples_leaf=0.015, min_samples_split=0.005, n_estimators=160; total time=   2.2s
[CV] END criterion=entropy, max_depth=35, max_features=log2, min_samples_leaf=0.035, min_samples_split=0.005, n_estimators=160; total time=   2.5s
[CV] END criterion=entropy, max_depth=35, max_features=log2, min_samples_leaf=0.035, min_samples_split=0.005, n_estimators=160; total time=   2.3s
[CV] END criterion=entropy, max_depth=35, max_features=log2, min_samples_leaf=0.035, min_samples_split=0.005, n_estimators=160; total time=   2.2s
[CV] END criterion=gini, max_depth=5, max_features=log2, min_sample

RandomizedSearchCV(cv=3,
                   estimator=RandomForestClassifier(class_weight='balanced'),
                   n_iter=100,
                   param_distributions={'criterion': ['gini', 'entropy'],
                                        'max_depth': [5, 15, 25, 35, None],
                                        'max_features': ['sqrt', 'log2'],
                                        'min_samples_leaf': [0.005, 0.015,
                                                             0.025, 0.035],
                                        'min_samples_split': [0.005, 0.015,
                                                              0.025, 0.035],
                                        'n_estimators': [10, 60, 110, 160,
                                                         210]},
                   return_train_score=True,
                   scoring=<function my_roc_auc_score at 0x7f817530c940>,
                   verbose=2)

In [26]:
model_rf.best_params_

{'n_estimators': 110,
 'min_samples_split': 0.005,
 'min_samples_leaf': 0.005,
 'max_features': 'sqrt',
 'max_depth': 35,
 'criterion': 'gini'}

In [27]:


df_cv_results = pd.DataFrame(model_rf.cv_results_)
df_cv_results = df_cv_results[['rank_test_score','mean_test_score','mean_train_score',
                           'param_n_estimators', 'param_min_samples_split','param_min_samples_leaf',
                           'param_max_features', 'param_max_depth','param_criterion']]
df_cv_results.sort_values('rank_test_score', inplace=True)
df_cv_results[:20]


rank_test_score  mean_test_score  mean_train_score param_n_estimators  \
53                1         0.845814          0.851212                110   
68                2         0.845319          0.851127                160   
84                3         0.841956          0.846489                210   
87                4         0.841898          0.848140                 10   
95                5         0.841268          0.845466                210   
59                6         0.841250          0.846532                 10   
79                7         0.839963          0.844945                210   
54                8         0.837440          0.842544                 10   
63                9         0.833966          0.837350                210   
74               10         0.833829          0.837065                160   
98               11         0.832533          0.835535                110   
48               12         0.832475          0.835183                160   
27               13         0.832193          0.836179                160   
58               14         0.831868          0.835794                110   
12               15         0.829995          0.835140                 10   
76               16         0.826689          0.828658                160   
64               17         0.826539          0.828799                160   
10               18         0.826432          0.829036                160   
43               19         0.824820          0.827268                 60   
38               20         0.824723          0.827866                 10   

   param_min_samples_split param_min_samples_leaf param_max_features  \
53                   0.005                  0.005               sqrt   
68                   0.005                  0.005               sqrt   
84                   0.015                  0.005               log2   
87                   0.005                  0.005               log2   
95                   0.015                  0.005               log2   
59                   0.005                  0.005               log2   
79                   0.015                  0.005               log2   
54                   0.015                  0.005               log2   
63                   0.025                  0.005               log2   
74                   0.025                  0.005               sqrt   
98                   0.025                  0.005               log2   
48                   0.025                  0.005               log2   
27                   0.025                  0.005               sqrt   
58                   0.025                  0.005               log2   
12                   0.025                  0.005               sqrt   
76                   0.035                  0.005               sqrt   
64                   0.035                  0.005               log2   
10                   0.005                  0.005               sqrt   
43                   0.005                  0.005               sqrt   
38                   0.035                  0.005               sqrt   

   param_max_depth param_criterion  
53              35            gini  
68              15         entropy  
84              15         entropy  
87            None         entropy  
95            None            gini  
59              35         entropy  
79              15            gini  
54              15            gini  
63            None         entropy  
74              35         entropy  
98              35            gini  
48              25            gini  
27              25            gini  
58              15            gini  
12              25         entropy  
76              25         entropy  
64              35         entropy  
10               5            gini  
43               5         entropy  
38              15            gini

In [28]:

df_cv_results.to_csv('./Data/random_search.csv', index=False)

In [33]:
# {'n_estimators': 110,
#  'min_samples_split': 0.005,
#  'min_samples_leaf': 0.005,
#  'max_features': 'sqrt',
#  'max_depth': 35,
#  'criterion': 'gini'}
model_rf_fin = RandomForestClassifier(class_weight='balanced',
                                      criterion='gini',
                                      max_depth=35,
                                      max_features='sqrt',
                                      min_samples_leaf=0.005,
                                      min_samples_split=0.005,
             
                                      n_estimators=110)
model_rf_fin.fit(x_train,y_train)


y_pred_train = model_rf_fin.predict(x_train)
y_pred_test = model_rf_fin.predict(x_test)
print("\n Training Accuracy score:",metrics.accuracy_score(y_train, y_pred_train))
print("Testing Accuracy score:",metrics.accuracy_score(y_test, y_pred_test))




 Training Accuracy score: 0.8495012175005892
Testing Accuracy score: 0.8532824203260391
